In [2]:
### mnist CNN

In [102]:
from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import *
from keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np
import sys
import tensorflow as tf
import os

In [103]:
(xtrain,ytrain), (xtest,ytest) = mnist.load_data()

In [104]:
xtrain = xtrain.reshape(xtrain.shape[0],28,28,1).astype('float64')/255
xtest = xtest.reshape(xtest.shape[0],28,28,1).astype('float64')/255

In [105]:
ytrain = np_utils.to_categorical(ytrain,10)
ytest = np_utils.to_categorical(ytest,10)

In [106]:
print(xtrain.shape, ytrain.shape, xtest.shape, ytest.shape)

(60000, 28, 28, 1) (60000, 10) (10000, 28, 28, 1) (10000, 10)


In [107]:
# model 구성
model = Sequential()
# Conv2D(filter갯수, kernel_size=(,), input_shape=(r,c,color), activation= )
model.add(Conv2D(32, kernel_size=(3,3), input_shape=(28,28,1), activation='relu'))
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.25))
# Flatten() - 1차원으로 변경
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [108]:
## model 환경 설정
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [109]:
# model 최적화
modeldir = './mycnnmodel/'

if not os.path.exists(modeldir):
    modelpath = './mycnnmodel/{epoch:02d}-{val_loss:.4f}.hdf5'
    os.mkdir(modeldir)
# model 저장 옵션 설정
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)

# 
es = EarlyStopping(monitor='val_loss', patience=10)

In [110]:
from keras.preprocessing.image import ImageDataGenerator

In [111]:
train_gen = ImageDataGenerator(rescale=1./255)
train_gen = train_gen.flow_from_directory('C:\\Users\\student\\Desktop\\dataset\\python\\image_data\\train', 
                             target_size=(24,24), batch_size=3, class_mode='categorical')

Found 45 images belonging to 3 classes.


In [112]:
test_gen = ImageDataGenerator(rescale=1./255)
test_gen = test_gen.flow_from_directory('C:\\Users\\student\\Desktop\\dataset\\python\\image_data\\test', 
                             target_size=(24,24), batch_size=3, class_mode='categorical')

Found 15 images belonging to 3 classes.


In [113]:
# 크기(24,24), 채널 3, 필터 3*3, 필터개수 32, relu
model=Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(24,24,3)))
# 필터 3*3, 필터개수 64, relu
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
# maxpool 2*2
model.add(MaxPooling2D(pool_size=(2,2)))
# 플래튼 => Dense(128개 뉴런 출력) => Dense(softmax)
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))

In [114]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 22, 22, 32)        896       
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 20, 20, 64)        18496     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 6400)              0         
_________________________________________________________________
dense_15 (Dense)             (None, 128)               819328    
_________________________________________________________________
dense_16 (Dense)             (None, 3)                 387       
Total params: 839,107
Trainable params: 839,107
Non-trainable params: 0
________________________________________________

In [115]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [116]:
model.fit_generator(train_gen, steps_per_epoch=15, epochs=50, validation_data=test_gen, validation_steps=5)

Epoch 1/50
15/15 [==============================] - 1s 70ms/step - loss: 0.8140 - accuracy: 0.6444 - val_loss: 0.2304 - val_accuracy: 0.9333
Epoch 2/50
15/15 [==============================] - 0s 29ms/step - loss: 0.1056 - accuracy: 0.9556 - val_loss: 0.0052 - val_accuracy: 0.9333
Epoch 3/50
15/15 [==============================] - 0s 32ms/step - loss: 0.0203 - accuracy: 1.0000 - val_loss: 0.2091 - val_accuracy: 1.0000
Epoch 4/50
15/15 [==============================] - 1s 34ms/step - loss: 0.0056 - accuracy: 1.0000 - val_loss: 0.0155 - val_accuracy: 0.9333
Epoch 5/50
15/15 [==============================] - 1s 34ms/step - loss: 7.7786e-04 - accuracy: 1.0000 - val_loss: 0.0014 - val_accuracy: 0.9333
Epoch 6/50
15/15 [==============================] - 1s 36ms/step - loss: 2.7077e-04 - accuracy: 1.0000 - val_loss: 3.1471e-05 - val_accuracy: 0.9333
Epoch 7/50
15/15 [==============================] - 1s 34ms/step - loss: 1.8724e-04 - accuracy: 1.0000 - val_loss: 0.0096 - val_accuracy: 1.00

In [123]:
model.evaluate_generator(test_gen, steps=5)[1]

1.0

In [118]:
model.predict_generator(test_gen, steps=5)

array([[6.8117572e-07, 9.9999928e-01, 6.8646683e-10],
       [9.9999654e-01, 2.2137046e-06, 1.1500473e-06],
       [8.8876398e-08, 9.9999988e-01, 1.1503687e-09],
       [4.3985364e-07, 7.3109054e-06, 9.9999225e-01],
       [9.9674428e-01, 1.6082970e-05, 3.2396787e-03],
       [3.6007477e-05, 1.2474626e-02, 9.8748940e-01],
       [2.0504087e-06, 9.9999785e-01, 7.6969521e-08],
       [1.3800776e-08, 1.6366633e-06, 9.9999833e-01],
       [9.9999952e-01, 3.4526209e-07, 1.0084351e-07],
       [4.7108729e-06, 9.9999523e-01, 3.2205250e-09],
       [3.5531835e-07, 1.4823418e-07, 9.9999952e-01],
       [9.9999714e-01, 2.6842101e-06, 8.6667605e-08],
       [2.9297391e-01, 5.6657231e-01, 1.4045385e-01],
       [9.9995375e-01, 4.2251031e-05, 4.1034159e-06],
       [3.5929490e-06, 1.6049069e-03, 9.9839145e-01]], dtype=float32)

In [119]:
from keras.preprocessing.image import array_to_img, img_to_array, load_img
re_gen = ImageDataGenerator(rescale=1./255, rotation_range=15, width_shift_range=0.1, height_shift_range=0.1,
                  shear_range=0.5, zoom_range=[0.8,2.0], horizontal_flip=True,  vertical_flip=True, fill_mode='nearest')

In [120]:
img = load_img('C:\\Users\\student\\Desktop\\dataset\\python\\image_data\\train\\triangle\\triangle001.png')

In [121]:
x = img_to_array(img)
x = x.reshape((1,)+x.shape)
x.shape

(1, 24, 24, 3)

In [122]:
i = 0
for batch in re_gen.flow(x,batch_size=1, save_to_dir='store', save_prefix='tri', save_format='png'):
    i += 1
    if i > 50:
        break